In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
final_df = pd.read_excel("final_data.xlsx")

# Extract features and target
X = final_df.drop(['result'], axis=1)
y = final_df['result']

# Replace inf and negative values in 'cur_run_rate' with zeros
X['cur_run_rate'] = X['cur_run_rate'].apply(lambda x: 0 if x <= 0 or np.isinf(x) else x)

# Encode categorical variables
label_encoders = {}
categorical_columns = ['Batting_team', 'Bowling_team', 'city']

for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    X[col] = label_encoders[col].fit_transform(X[col])

# Split the data into training (70%), validation (15%), and testing (15%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=45)
X_validate, X_test, y_validate, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=45)

# Define the parameter grid to search over
param_grid = {
    'n_estimators': [100],
    'max_depth': [10],
    'min_samples_split': [2],
    'min_samples_leaf': [2]
}

# Create and train the RandomForestClassifier with GridSearchCV
clf = RandomForestClassifier(random_state=45)
grid_search = GridSearchCV(clf, param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Make predictions on the validation set using the best model
best_clf = grid_search.best_estimator_
validate_pred = best_clf.predict(X_validate)

# Calculate and print the validation accuracy
accuracy_validate = accuracy_score(y_validate, validate_pred)
print("Validation Accuracy:", accuracy_validate)

# Print a classification report for more detailed performance metrics
print("Validation Classification Report:")
print(classification_report(y_validate, validate_pred))

# Get the confusion matrix for the validation set
confusion_matrix_validate = confusion_matrix(y_validate, validate_pred)
print("Validation Confusion Matrix:")
print(confusion_matrix_validate)

# Plot the confusion matrix as a chart
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_validate, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Validation Confusion Matrix')
plt.show()

# Make predictions on the test set using the best model
test_pred = best_clf.predict(X_test)

# Calculate and print the test accuracy
accuracy_test = accuracy_score(y_test, test_pred)
print("Test Accuracy:", accuracy_test)

# Print a classification report for more detailed performance metrics
print("Test Classification Report:")
print(classification_report(y_test, test_pred))

# Get the confusion matrix for the test set
confusion_matrix_test = confusion_matrix(y_test, test_pred)
print("Test Confusion Matrix:")
print(confusion_matrix_test)

# Plot the confusion matrix as a chart for the test set
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_test, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Test Confusion Matrix')
plt.show()
